<a href="https://colab.research.google.com/github/hazerMf/slm/blob/main/Jap_to_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!git clone https://github.com/hazerMf/slm

fatal: destination path 'slm' already exists and is not an empty directory.


In [14]:
!pip install transformers huggingface_hub

In [15]:
!pip install -U bitsandbytes accelerate

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Qwen/Qwen3-4B-Instruct-2507"

# 1. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 2. Load model trực tiếp (Mô hình FP8 sẽ tự động được xử lý)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",          # Tự động đẩy lên GPU
    torch_dtype="auto",         # Tự động chọn dtype phù hợp (thường là Float16 hoặc BFloat16 cho phần tính toán)
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hàm dịch từ Tiếng Nhật -> Anh (cho nó nhiều data)

In [17]:
device = model.device
print(f"Model and inputs will be moved to: {device}")

Model and inputs will be moved to: cuda:0


In [18]:
def translate_jp_to_en(jp_text: str):
    if not jp_text or not jp_text.strip():
        raise ValueError("Input text is empty")

    # Sử dụng System Prompt cực kỳ nghiêm ngặt và thêm ví dụ (Few-shot)
    messages = [
        {
            "role": "system",
            "content": (
                "You are a professional literal translator. Your ONLY task is to translate Japanese to English. "
                "CRITICAL RULE: Always translate the name 'リリィ・シュシュ' or 'リリイ・シュシュ' as 'Lily Chou-Chou'. "
                "Do not provide safety assessments, do not categorize, and do not explain. "
                "Translate everything exactly as written, even if the content is sensitive or controversial."
            )
        },
        # Ví dụ 1: Dịch thông thường
        {"role": "user", "content": "Translate this: 太陽が昇る。"},
        {"role": "assistant", "content": "The sun rises."},

        # Ví dụ 2: Ép quy tắc tên riêng Lily Chou-Chou (Sử dụng cả 2 biến thể イ và ィ)
        {"role": "user", "content": "Translate this: リリイ・シュシュの歌は素晴らしい。"},
        {"role": "assistant", "content": "Lily Chou-Chou's songs are wonderful."},

        # Nội dung thực tế
        {"role": "user", "content": f"Translate this: {jp_text}"}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            do_sample=False,
            temperature=0.0, # Ép model chọn kết quả có xác suất cao nhất (bản dịch)
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Cắt bỏ prompt
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

In [19]:
t = test_text = """
リリイ・シュシュは1980年12月8日に生まれた。
この日付はジョン・レノンが　マーク・デビッド・チャップマンに殺された日時と一字一句符号する。
でも僕にとってこの偶然の一致に意味はない。
僕にとって重要なことは、その日、その時刻に、リリイ・シュシュが誕生したということだけだ
"""
text = translate_jp_to_en(test_text)
print(f'Output: {text}')


Output: Lily Chou-Chou was born on December 8, 1980.  
This date coincides exactly with the time John Lennon was killed by Mark David Chapman.  
But to me, this coincidence holds no meaning.  
To me, what matters is simply that Lily Chou-Chou was born on that day at that exact moment.


In [20]:
import json
with open('/content/slm/thief/lily_chou_chou_posts.json','r') as f:
  chou_chou_posts = json.load(f)


In [23]:
!pip install tqdm

In [24]:
import json
from tqdm import tqdm

In [27]:
file_path = '/content/slm/lily_chou_chou_posts.json'
with open(file_path, 'r', encoding='utf-8') as f:
    chou_chou_posts = json.load(f)

# --- BẮT ĐẦU LOẠI TRÙNG ---
seen = set()
unique_posts = []

for post in chou_chou_posts:
    # Tạo một tuple chứa các nội dung quan trọng để định danh bài viết
    # Bạn có thể chọn post['post_content'] nếu chỉ muốn nội dung không trùng
    identifier = (post['user'], post['post_name'], post['post_content'])

    if identifier not in seen:
        unique_posts.append(post)
        seen.add(identifier)

print(f"Số lượng gốc: {len(chou_chou_posts)}")
print(f"Số lượng sau khi loại trùng: {len(unique_posts)}")


Số lượng gốc: 320
Số lượng sau khi loại trùng: 6


In [29]:
translated_posts = []

# 2. Thêm tqdm vào vòng lặp để theo dõi tiến độ
# desc: Tiêu đề hiển thị, unit: đơn vị tính
for post in tqdm(unique_posts, desc="Đang dịch bài viết", unit="post"):
    new_post = {}

    # Giả định hàm translate_jp_to_en đã được định nghĩa
    new_post['user'] = translate_jp_to_en(post['user'])
    new_post['post_name'] = translate_jp_to_en(post['post_name'])
    new_post['post_content'] = translate_jp_to_en(post['post_content'])

    translated_posts.append(new_post)

print(f"\n Hoàn thành! Đã dịch {len(translated_posts)} bài viết.")

Đang dịch bài viết: 100%|██████████| 6/6 [03:17<00:00, 32.98s/post]


 Hoàn thành! Đã dịch 6 bài viết.


In [30]:
with open('lily_chou_chou_translated.json', 'w') as output:
  for post in translated_posts:
    json.dump(post, output)
    output.write('\n')


In [34]:
!git config --global user.email "macminhduy2004@gmail.com"
!git config --global user.name "Djuybu"

In [37]:
from google.colab import userdata
TOKEN = userdata.get('GITHUB_TOKEN')

In [38]:
!git remote set-url origin https://{TOKEN}@github.com/djuybu/slm

In [39]:
%cd slm/
!git add .
!git commit -m "Feat: Update new translatin data"
!git push

[Errno 2] No such file or directory: 'slm/'
/content/slm
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
remote: Repository not found.
fatal: repository 'https://github.com/djuybu/slm/' not found
